# 遺伝的アルゴリズムによるD最適化計画法

`deap`という遺伝的アルゴリズムのパッケージを利用する。

D最適化基準はデータ行列$X$に対して行列式$|X^TX|$で与えられる。

触媒データを利用してサンプル選定を行う。


In [0]:
!wget https://raw.githubusercontent.com/funatsu-lab/support-page/master/data/catalyst/journal_data.csv
!ls journal_data.csv

--2020-01-11 13:24:44--  https://raw.githubusercontent.com/funatsu-lab/support-page/master/data/catalyst/journal_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5502 (5.4K) [text/plain]
Saving to: ‘journal_data.csv’

journal_data.csv    100%[===================>]   5.37K  --.-KB/s    in 0s      

2020-01-11 13:24:49 (81.6 MB/s) - ‘journal_data.csv’ saved [5502/5502]

journal_data.csv


In [0]:
# deapのインストール 
!pip install deap

     |████████████████████████████████| 153kB 9.5MB/s 


In [0]:
from pandas import read_csv
from pandas import DataFrame
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from random import randint
from deap import creator, base, tools, algorithms
from numpy import log as np_log, where as np_where, array as np_array, arange as np_arange, exp as np_exp, unique as np_uniq, power as np_pow
from numpy.linalg import det as np_det
from numpy.random import permutation as np_perm
from tqdm import tqdm
from pprint import pprint 

In [0]:
df = read_csv('./journal_data.csv', header=0, index_col=0)

In [0]:
df_train, df_test = train_test_split(df, test_size=.33, random_state=66)
print(df_train.shape, df_test.shape)

(50, 23) (25, 23)


In [0]:
INPUT=df_train.columns[:-3].tolist()

In [0]:
Xtrain = scale(df_train[INPUT])
print('determinant', np_det(Xtrain.T @ Xtrain))
print('scaled determinant', np_det(Xtrain.T @ Xtrain)**(1/len(INPUT)))

determinant 1.0109311742906108e+27
scaled determinant 22.399384226277576


## トレーニングデータからD最適化計画を取得する

50個から17サンプルを取り出す事を考える。<br>
すると${}_{50}C_{17}$ 通りも調べないと行けなくなり、効率が悪い。<br>
そこで、最適化手法の1つである遺伝的アルゴリズムを利用する。

In [0]:
def d_criterion(X):
  return np_log(np_pow(np_det(X.T @ X), (1/X.shape[1])) )

In [0]:
d_criterion(Xtrain)

3.1090334685848173

## Deapを用いた実装
[公式サンプルコード](https://github.com/DEAP/deap)

In [0]:
def perm(n_max, n_out):
  return np_perm(np_arange(n_max)).tolist()[:n_out]
print(len(perm(50, 17)))
print(perm(50, 17))

17
[24, 16, 20, 18, 37, 45, 2, 31, 25, 22, 38, 9, 44, 6, 19, 27, 7]


In [0]:
n_pop = 300
n_dim = 20
n_gen=  100
n_samples=Xtrain.shape[0]

In [0]:
def d_opt01(individual):
  "Deap evaluation function. "
  if sum(individual)!=n_dim:
    return -9999.99,
  x_in = np_array(individual)
  x_sc = Xtrain[np_where(x_in==1)[0], :]
  return d_criterion(x_sc),

In [0]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_index", randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_index, n=n_samples)
toolbox.register("population", tools.initRepeat, 
                 list, toolbox.individual)
toolbox.register("evaluate", d_opt01)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
population = toolbox.population(n=n_pop)

In [0]:
for gen in tqdm(range(n_gen)):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in power
  
100%|██████████| 100/100 [00:05<00:00, 19.12it/s]


In [0]:
ret10 = list(map(toolbox.evaluate, top10))
print(top10)  # D最適化基準のトップ10に対応したサンプルの要(1)/不要(0)の数列。
pprint(ret10) # D最適化基準のトップ10の値

[[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,

**トップ10の確認**。<br>
全て揃っていることが分かり、最適化に成功している。

In [0]:
for top in top10:
  print(top)

[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 

**D最適化基準を標準化する場合**

$d$次元のデータ行列$X$に対して以下のように定めることもある。

\begin{equation}
D = \det|X^TX|^{1/d}
\end{equation}



In [0]:
xin = np_array(top10[0])
mat_ = Xtrain[np_where(xin==1)[0], :]
print('D criterion', np_pow(np_det(mat_.T@mat_), 1./mat_.shape[1]))

D criterion 8.602635504346852
